### NREL Solar Resource Data - lat/long file import

#### This program uses the Solar Resource Data API provided by NREL and returns various solar power related parameters for a given latitude and longitude.  The goal of the program is to take in a file of lat and long values and iterate through, pulling the values of interest and then writing them to a new file in a format usable for creation of maps and analytical purposes.  


##### **DATA RETURNED**  
Average Direct Normal Irradiance - measured surface perpendicular to direct solar rays (avg_dni)  
Average Global Horizontal Irradiance - measured surface horizontal to the ground (avg_ghi)  
Average Tilt at Latitude (avg_lat_tilt)  


##### **INFORMATION & RESEARCH**
https://developer.nrel.gov/docs/solar/solar-resource-v1/   
https://en.wikipedia.org/wiki/Solar_irradiance  
https://www.3tier.com/en/support/solar-online-tools/what-global-horizontal-irradiance-solar-prospecting/  
https://www.pveducation.org/pvcdrom/properties-of-sunlight/measurement-of-solar-radiation   


In [2]:
import pandas as pd
import csv
import requests, json, csv

# name of csv with the lat/long and location information for the queries
# column headers for lat and long must be named as such
coordinate_file = 'location_values.csv'

In [3]:
# review list of lat longs to use for reading into API
with open(coordinate_file, newline='') as csvfile:
    loc_reader = csv.reader(csvfile)
    for loc in loc_reader:
        print(loc, end="\n")

['ï»¿City', 'State', 'lat', 'long']
['Montgomery', ' AL', '32.361668', '-86.279167']
['Auburn', ' AL', '32.609856', '-85.480782']
['Tuscaloosa', ' AL', '33.189281', '-87.565155']
['Mobile', ' AL', '30.695366', '-88.039894']
['Hot Springs', ' AR', '34.496212', '-93.05722']
['Fayetteville', ' AR', '36.082157', '-94.171852']
['Jonesboro', ' AR', '35.835575', '-90.705025']
['Surprise', ' AZ', '33.630554', '-112.366669']
['Yuma', ' AZ', '32.698437', '-114.650398']
['Peoria', ' AZ', '33.580944', '-112.237068']
['Tempe', ' AZ', '33.427204', '-111.939896']
['Phoenix', ' AZ', '33.448376', '-112.074036']
['Flagstaff', ' AZ', '35.198284', '-111.651299']
['Port Hueneme', ' CA', '34.155834', '-119.202789']
['Vista', ' CA', '33.193611', '-117.241112']
['Berkeley', ' CA', '37.871666', '-122.272781']
['Pasadena', ' CA', '34.156113', '-118.131943']
['Escondido', ' CA', '33.124722', '-117.080833']
['Modesto', ' CA', '37.661388', '-120.994446']
['Irvine', ' CA', '33.669445', '-117.823059']
['Palm Springs

In [4]:
# bring lat / long csv into a dataframe
file_name = coordinate_file
df = pd.read_csv(file_name) 
df 

,City,State,lat,long
0,Montgomery,AL,32.361668,-86.279167
1,Auburn,AL,32.609856,-85.480782
2,Tuscaloosa,AL,33.189281,-87.565155
3,Mobile,AL,30.695366,-88.039894
4,Hot Springs,AR,34.496212,-93.057220
...,...,...,...,...
161,Green Bay,WI,44.513332,-88.015831
162,Racine,WI,42.722702,-87.784225
163,Brookfield,WI,43.061615,-88.107948
164,Huntington,WV,38.413651,-82.446732


### MAKE REQUESTS AND SAVE TO A FILE FOR ANALYSIS

In [5]:
# file name for saving all results
all_filename = input("filename for all data including .csv: ")

filename for all data including .csv: all_data.csv


In [6]:
# file name for saving yearly results
yearly_filename = input("filename for yearly data including .csv: ")

filename for yearly data including .csv: yearly_data.csv


In [8]:
# iterate through csv and query using lat / long to get the data and save to a csv
# create URL using values in database
# initiate counters to start at first index value in the lat / long file
lat_count = 0
long_count = 0
while lat_count < len(df):
    lat = df.at[lat_count,'lat']
    long = df.at[long_count,'long']
    API_ENDPOINT = 'https://developer.nrel.gov/api/solar/solar_resource/v1.json?api_key=aO3yoslbHz6Pg9vrVdVGjA47Jrv1TnjOfOhDJj0L&lat=%s&lon=%s' % (lat,long)
    full_url = API_ENDPOINT
    print()
    print('URL: ', full_url)    

    # send out the request for information
    response = requests.get(full_url)
    # check its status to see if all is well
    print('Made request, response status: ', response.status_code)
    # determine if results were returned or another error code
    if response.status_code == 200:
        # 200 is all good
        results = json.loads(response.text)
    elif response.status_code == 429:
        # 429 is request limit reached
        print()
        print("You have reached your hourly request limit.")
        break
    else: 
        # other exception noted on NREL is 422 for invalid parameter; the loop will continue to the next lat/long
        print()
        print("One of your input parameters is not valid.")

    # append all to a file
    with open(all_filename, 'a', newline="") as file_name:
        try:
            write_file = csv.writer(file_name, lineterminator="\r")
            for k, v in results['inputs'].items():
                write_file.writerow([k, v])
            for key, value in results['outputs'].items():
                write_file.writerow([key, value])
        except KeyError:
            print()
            print("Error with saving to", all_filename, ". You have requested an invalid value.")
            break

    # append value of interest to the yearly file; in this case it is 'avg_dni'
    with open(yearly_filename, 'a') as print_file:
        try:
            print_file.write(str(lat) + "\n")
            print_file.write(str(long) + "\n")
            print_file.write(str(results['outputs']['avg_dni']['annual'])) # result needs to be a string, not integer
            print_file.write("\n")
        except KeyError:
            print()
            print("Error with saving to", yearly_filename, ". You have requested an invalid value.")
            break

    # add 1 to lat / long counters to move to next values for next iteration
    lat_count = lat_count + 1
    long_count = long_count + 1
else:
    # notification that you've gone through all the locations in the csv file
    print()
    print("You've reached the end of the file.")


URL:  https://developer.nrel.gov/api/solar/solar_resource/v1.json?api_key=aO3yoslbHz6Pg9vrVdVGjA47Jrv1TnjOfOhDJj0L&lat=32.361668&lon=-86.279167
Made request, response status:  429

You have reached your hourly request limit.


### FORMAT YEARLY VALUES AND SAVE TO A FILE FORMATTED FOR GIS ANALYSIS

### FORMAT YEARLY VALUES AND SAVE TO A FILE FORMATTED FOR GIS ANALYSIS

In [ ]:
# write the yearly request results to a dataframe; note it's in a single column
result_file = yearly_filename
result_df = pd.read_csv(result_file, header=None) 
result_df 

In [ ]:
# rearrange values in file to better format and add header info
# ValueError when query is interrupted and a value isn't returned for the last lat/long and not enough rows for proper reformatting
try:
    df_new = (pd.DataFrame(result_df.values.reshape(-1, 3), 
                    columns=['lat','long','avg_yrly_dni']))
    print(df_new)
except ValueError:
    print()
    print("Your dataframe is not the appropriate length for reformatting - check for missing values.")

In [ ]:
# merge original dataframe with city and state with resultant, reformatted dataframe
df_all = pd.merge(df, df_new, on=['lat','long'], how='left')
df_all

In [ ]:
# write dataframe to a new csv
final_file = input("name of reformatted request result file, include .csv: ")
df_all.to_csv(final_file, index=False)